In [ ]:
# Import TensorFlow and Keras for building Neural Networks
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential  # For creating a linear stack of layers for our Neural Network
from keras.layers import Dense  # For creating regular densely-connected NN layers
from keras import Input  # For instantiating a keras tensor
import keras.metrics as metrics
from keras import backend as K
from keras.callbacks import LearningRateScheduler

# Import sklearn for model evaluation
import sklearn

## Create and train a neural network

In [ ]:
def create_model(network):
    """
    Create a neural network model for the TD_ANN_model ode system.
    
    Args:
    - network: A list representing the architecture of the neural network,
               e.g., [input_size, hidden_size1, hidden_size2, ..., output_size]
    
    Returns:
    - model: A Keras Sequential model.
    """
    
    model = Sequential()  # Initialize a Sequential model

    # Input Layer
    model.add(Input(shape=(network[0],), name='Input-Layer'))  # Define the input layer

    # Hidden Layers
    for i in range(1, len(network) - 1):
        model.add(Dense(network[i],
                        activation='softplus',  # Softplus activation function
                        use_bias=True,
                        kernel_initializer=keras.initializers.RandomUniform(minval=-1., maxval=1.),
                        bias_initializer=keras.initializers.RandomUniform(minval=-1., maxval=1.),
                        trainable=True,
                        name="Hidden-Layer-" + str(i)))

    # Output Layer
    model.add(Dense(network[-1],
                    activation='linear',  # Linear activation function
                    use_bias=False,
                    kernel_initializer=keras.initializers.RandomUniform(minval=-1., maxval=1.),
                    trainable=True,
                    name='Output-Layer'))

    return model


In [ ]:
def train_nn(model, X, y, epochs, batch_size, verbose='auto'):
    """
    Compile and train a Keras neural network model with the given data.

    Args:
    - model: The Keras model to be trained.
    - X: Input data.
    - y: Target data.
    - epochs: Number of training epochs (iterations over the dataset).
    - batch_size: Number of samples per gradient update.
    - verbose: Controls the verbosity of training. 'auto' (default) uses Keras's default verbosity settings.

    Returns:
    - model: The trained Keras model.
    """
    
    # Compile the Keras model
    model.compile(
        optimizer='sgd',  # Stochastic Gradient Descent optimizer
        loss="mean_squared_error",  # Loss function to be minimized (for regression problems)
        metrics=[metrics.mae]  # List of metrics to be evaluated during training
    )

    # Fit the Keras model on the dataset
    model.fit(
        X,  # Input data
        y,  # Target data
        batch_size=batch_size,  # Number of samples per batch
        epochs=epochs,  # Number of training epochs
        verbose=verbose  # Controls training verbosity
    )

    return model


In [ ]:
def get_stats(model, flag_summary=True):
    """
    Display summary and weights/biases information for a Keras model.

    Args:
    - model: The Keras model for which you want to display statistics.
    - flag_summary: If True, display model summary (default: True).

    Returns:
    - None
    """
    
    if flag_summary:
        print("")
        print('-------------------- Model Summary --------------------')
        model.summary()  # Display the model summary
        print("")

    print('-------------------- Weights and Biases --------------------')
    for layer in model.layers:
        print("Layer: ", layer.name)  # Print the layer name

        # Check if the layer has weights
        if layer.get_weights():
            # Display layer's weights (kernels)
            print("  --Kernels (Weights): ")
            print(layer.get_weights()[0])

            # Check if the layer has biases
            if len(layer.get_weights()) > 1:
                # Display layer's biases
                print("  --Biases: ")
                print(layer.get_weights()[1])
            else:
                print("  -- No biases")
        else:
            print("  -- No trainable weights")

        print("")

    print("")

    
    return

In [ ]:
def print_train_vars(model):
    """
    Print the names and values of trainable variables in the Keras model.

    Args:
    - model: The Keras model for which you want to print trainable variables.

    Returns:
    - None
    """
    for var in model.trainable_variables:
        print(f"{var.name}:\t {var.numpy().flatten()}")
        
    return

def print_grads(model, grads):
    """
    Print gradients for trainable variables in the Keras model.

    Args:
    - model: The Keras model.
    - grads: Gradients computed during training.

    Returns:
    - None
    """
    print("Gradients")
    for var, g in zip(model.trainable_variables, grads):
        print(f"{var.name}:\t {g.numpy().flatten()}")

    return
        
def print_before(model, grads):
    """
    Print trainable variables and gradients before training.

    Args:
    - model: The Keras model.
    - grads: Gradients computed before training.

    Returns:
    - None
    """
    print("=="*20)
    print("Trainable variables (before)")
    print_train_vars(model)
    print("--"*20)

    print("--"*40)
    print_grads(model, grads)
    print("--"*40)

    return
    
def print_after(model, grads):
    """
    Print trainable variables and gradients after training.

    Args:
    - model: The Keras model.
    - grads: Gradients computed after training.

    Returns:
    - None
    """
    print("--"*20)
    print("Trainable variables (after)")
    print_train_vars(model)
    print("=="*20)
    
    return
